In [2]:
import sys
import tyssue
import random
import numpy as np
import pandas as pd
import numpy as np
import json
import matplotlib.pylab as plt
%matplotlib inline
import ipyvolume as ipv
from matplotlib import pyplot
#from tyssue.solvers import QSSolver

from IPython.display import Image


from tyssue.topology.base_topology import add_vert


from tyssue import config, Sheet, SheetGeometry, History, EventManager, PlanarGeometry
from tyssue import SheetGeometry as geom
from tyssue import PlanarGeometry as geom
from tyssue.draw.plt_draw import create_gif
from tyssue.draw.plt_draw import plot_forces
from tyssue.draw import sheet_view
from tyssue.dynamics import effectors, model_factory
from tyssue.dynamics import PlanarModel
from tyssue.dynamics import SheetModel as model
from tyssue.generation import three_faces_sheet
from tyssue.solvers.viscous import EulerSolver
from tyssue.solvers.quasistatic import QSSolver
from tyssue.draw import sheet_view
from tyssue.draw.plt_draw import plot_forces
from tyssue.io import hdf5
import copy

from tyssue.geometry.planar_geometry import PlanarGeometry as geom
from tyssue.solvers.quasistatic import QSSolver
from tyssue.dynamics.planar_vertex_model import PlanarModel as model


from tyssue.stores import load_datasets

from sheet_topology import remove_face, cell_division

import warnings
warnings.filterwarnings('ignore')

### T1 transitions

from tyssue.topology.sheet_topology import type1_transition
from tyssue.topology.base_topology import collapse_edge, remove_face
from tyssue.topology.sheet_topology import split_vert as sheet_split
from tyssue.topology.bulk_topology import split_vert as bulk_split
from tyssue.topology import condition_4i, condition_4ii

ModuleNotFoundError: No module named 'tyssue.topology'

In [ ]:
def length_elasticity_range(cellmap, FRC_lower_length_elasticity, FRC_higher_length_elasticity, ECM_lower_length_elasticity, ECM_higher_length_elasticity):
    
    cellmap.edge_df['length_elasticity_ECM'] = cellmap.edge_df['length_elasticity']
    cellmap.edge_df['length_elasticity_FRC'] = cellmap.edge_df['length_elasticity']
        
    for edge in range(len(cellmap.edge_df)):
        FRC = random.randrange(FRC_lower_length_elasticity, FRC_higher_length_elasticity)/10000
        ECM = random.randrange(ECM_lower_length_elasticity, ECM_higher_length_elasticity)/10000
        cellmap.edge_df['length_elasticity'][edge] = FRC + ECM
        cellmap.edge_df['length_elasticity_ECM'][edge] = ECM
        cellmap.edge_df['length_elasticity_FRC'][edge] = FRC
    return cellmap

In [ ]:
def prefered_area_range(cellmap, lower_prefered_area, higher_prefered_area):
    for face in range(len(cellmap.face_df)):
        newValue = random.randrange(lower_prefered_area, higher_prefered_area)/10000
        cellmap.face_df['prefered_area'][face] = newValue
    return cellmap

In [ ]:
def change_specs(cellmap, geom, model, history, area_elasticity,
                 lower_prefered_area, higher_prefered_area, FRC_lower_length_elasticity, FRC_higher_length_elasticity, ECM_lower_length_elasticity,
                 ECM_higher_length_elasticity, line_nought):
    
    
    length_elasticity_range(cellmap, FRC_lower_length_elasticity, FRC_higher_length_elasticity, ECM_lower_length_elasticity, ECM_higher_length_elasticity)
    prefered_area_range(cellmap, lower_prefered_area, higher_prefered_area)
    
    cellmap.face_df["area_elasticity"] = area_elasticity
    cellmap.edge_df["prefered_length"] = line_nought


    cellmap.vert_df["viscosity"] = 1   
    
    #cellmap_original.face_df["prefered_area"][0] = 2
    #cellmap_original.face_df["prefered_area"][41] = 2

    
    solver = QSSolver()
    res = solver.find_energy_min(cellmap, geom, model)

    fig, ax = sheet_view(cellmap, mode="quick")
    fig.set_size_inches(10, 10)
    

        

    
    cellmap_new = copy.deepcopy(cellmap_original)
    geom_new = copy.deepcopy(geom_original)
    model_new = copy.deepcopy(model_original)
    history_new = copy.deepcopy(history_original)
    
    return [cellmap_new, geom_new, model_new, history_new]

In [ ]:
## Definition of the sheet

cellmap_original = Sheet.planar_sheet_2d('cellmap', 20, 20, 1, 1) # This may need to change
geom_original  = PlanarGeometry

to_cut = cellmap_original.cut_out([(-2, 10), (-2, 10)])
#cellmap_original.remove(to_cut, trim_borders=True)
cellmap_original.sanitize(trim_borders=True)
geom.center(cellmap_original)
geom.update_all(cellmap_original)
cellmap_original.update_rank()

model_original = model_factory([    
    effectors.FaceAreaElasticity,
    effectors.LengthElasticity
    ])

history_original = History(cellmap_original,
                           extra_cols={"edge":["dx", "dy"]})

cellmap_original.update_specs(model_original.specs)

fig, ax = sheet_view(cellmap_original, ['x', 'y'], mode='quick')

In [ ]:
[cellmap_H, geom_H, model_H, history_H] = change_specs(cellmap_original, geom_original, model_original, history_original, area_elasticity = 0.5, lower_prefered_area = 1000, higher_prefered_area = 10000, 
                                                       FRC_lower_length_elasticity = 1000, FRC_higher_length_elasticity = 10000, ECM_lower_length_elasticity = 100, 
                                                       ECM_higher_length_elasticity = 1000, line_nought = 0.2)

In [ ]:
cellmap_original.edge_df

In [ ]:
cellmap_original.face_df

In [ ]:
draw_specs = tyssue.config.draw.sheet_spec()

draw_specs['vert']['visible'] = True

draw_specs['vert']['color'] = "red"
draw_specs['vert']['alpha'] = 0.5
draw_specs['vert']['s'] = 5
coords = ['x', 'y']
fig, ax = sheet_view(cellmap_original, coords, **draw_specs)
fig.set_size_inches((10, 10))

In [ ]:
new_verts = np.empty((0, 3))
for numEdge in range(len(cellmap_original.edge_df)):
    currentEdge = cellmap_original.edge_df.loc[numEdge];
    if currentEdge['length']>0.7:
        new_vert = add_vert(cellmap_original, numEdge)
        new_verts = np.append(new_verts, [new_vert])

In [ ]:
new_verts

In [ ]:
draw_specs = tyssue.config.draw.sheet_spec()

draw_specs['vert']['visible'] = True

draw_specs['vert']['color'] = "red"
draw_specs['vert']['alpha'] = 0.5
draw_specs['vert']['s'] = 5
coords = ['x', 'y']
fig, ax = sheet_view(cellmap_original, coords, **draw_specs)
fig.set_size_inches((10, 10))

In [ ]:
fig, ax = sheet_view(cellmap_original)
fig.set_size_inches(8, 8)

for vert, data in cellmap_original.vert_df.iterrows():
    ax.text(data.x, data.y+0.02, vert, weight="bold", color="blue")

In [ ]:
fig, ax = sheet_view(cellmap_original)
fig.set_size_inches(8, 8)

for face, data in cellmap_original.face_df.iterrows():
    ax.text(data.x, data.y+0.02, vert, weight="bold", color="blue")

In [ ]:
cellmap_original.vert_df

In [ ]:
vertex1 = cellmap_original.vert_df.loc[139] 
vertex2 = cellmap_original.vert_df.loc[106] 

new_edge = {
    "srce": vertex1,
    "trgt": vertex2,
    "face": 140,  # Assuming the edge is part of the first face
}

# Append the new edge to the edge data frame
cellmap_original.edge_df = cellmap_original.edge_df.append(new_edge, ignore_index=True)

In [ ]:
draw_specs = tyssue.config.draw.sheet_spec()

draw_specs['vert']['visible'] = True

draw_specs['vert']['color'] = "red"
draw_specs['vert']['alpha'] = 0.5
draw_specs['vert']['s'] = 5
coords = ['x', 'y']
fig, ax = sheet_view(cellmap_original, coords, **draw_specs)
fig.set_size_inches((10, 10))

In [ ]:
import logging
import warnings

import numpy as np
import pandas as pd

from tyssue.topology.base_topology import add_vert, close_face, collapse_edge, remove_face
from tyssue.topology.base_topology import split_vert as base_split_vert

In [ ]:
daughter = cell_division(cellmap_original, 22, geom_original, angle=np.pi/2)
monolayer.cell_df.loc[[mother, daughter], 'prefered_area'] /= 2

solver = QSSolver()
res = solver.find_energy_min(cellmap_original, geom_original, model_original)
print(res['success'])

fig, ax = sheet_view(cellmap_original)
fig.set_size_inches(12, 5)

In [ ]:
fig, ax = sheet_view(cellmap_original)
fig.set_size_inches(8, 8)

for face, data in cellmap_original.face_df.iterrows():
    ax.text(data.x, data.y, face, fontsize=14, color="r")

In [ ]:
cellmap_original.